In [ ]:
import os
import time
import numpy as np
from numpy.linalg import matrix_power
import h5py
from fretbursts import *

sns = init_notebook

# H2MM processing

In [ ]:
class h2mm_model:
    def __init__(self,prior,trans,obs,loglik=-np.inf):
        self.nstate = prior.size
        self.nchannels = obs.shape[1]
        if self.nstate == prior.shape[0] == trans.shape[1] == trans.shape[0] == obs.shape[0]:
            self.prior = prior
            self.obs = obs
            self.trans = trans
            self.loglik = loglik
        else:
            print('Error: prior, trans and obs must have same number of states')
            self.prior = prior
            self.obs = obs
            self.trans = trans
            self.loglik = loglik
    def normalize(self):
        self.prior = self.prior / np.sum(self.prior)
        for i in range(0,self.nstate):
            self.trans[i,:] = self.trans[i,:] / np.sum(self.trans[i,:])
            self.obs[i,:] = self.obs[i,:] / np.sum(self.obs[i,:])
            
class fwdback_values:
    def __init__(self,alpha,beta,gamma,loglik,xi_summed,scale):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.loglik = loglik
        self.xi_summed = xi_summed
        self.scale = scale


In [ ]:
class ph_factors:
    def __init__(self,deltas):
        deltas = np.unique(deltas)
        deltas = deltas[deltas!=0]
        if deltas[0] != 1:
            deltas = np.append(1,deltas)
        self.deltas = deltas
        self.R = self.factorize(deltas)
        self.expand()
    def factorize(self,deltas):
        deltas = np.unique(deltas)
        deltas = deltas[deltas!=0]
        if deltas[0] != 1:
            deltas = np.append(1,deltas)
        R = [np.array([[0,1]])]
        for i in range(1,deltas.size):
            d = deltas[i] // deltas[i-1]
            m = deltas[i] % deltas[i-1]
            R.append(np.array([[i-1,d]]))
            while m != 0:
                k = np.argwhere(deltas == m)
                if k.size != 0:
                    R[i] = np.append(R[i],np.array([[k[0,0], 1]]),axis=0)
                    m = 0
                else:
                    k = np.argwhere(deltas < m)[-1,0]
                    d = m // deltas[k]
                    m = m % deltas[k]
                    R[i] = np.append(R[i],np.array([[k,d]],dtype=int),axis=0)
        return R
    def expand(self):
        B = np.array([],dtype=int)
        for i in range(0,len(self.R)):
            if self.R[i].shape[0] > 1:
                k = np.argwhere(self.R[i][:,1]>1)
                for j in range(0,k.shape[0]):
                    temp_ind = self.R[i][k[j,0],0]
                    temp_d = self.R[i][k[j,0],1]
                    temp_t = self.deltas[temp_ind]
                    temp_add = temp_t*temp_d
                    B = np.append(B,temp_t*temp_d)
        B = np.unique(B)
        deltas = np.append(self.deltas,B)
        deltas = np.unique(deltas)
        if deltas.size != self.deltas.size:
            self.deltas = deltas
            self.R = self.factorize(deltas)
            S = True
        else:
            S = False
        return S

In [ ]:
def CalculatePowerofTransMatrices(h_model,R):
    nfact = len(R)
    nstat = h_model.nstate
    M = np.zeros((nfact,h_model.nstate,h_model.nstate))
    M[0,:,:] = h_model.trans
    for ii in range(1,nfact):
        M_temp = np.eye(h_model.nstate)
        for iii in range(0,R[ii].shape[0]):
            M[ii,:,:] = M_temp @ matrix_power(M[R[ii][iii,0],:,:],R[ii][iii,1])
            M_temp = M[ii,:,:]
    return M

In [ ]:
def Rho_product_fast(A_dt1,A_dt2,Rho1,Rho2):
    N = Rho1.shape[0]
    Rho12 = np.zeros((N,N,N,N))
    for m in range(0,N):
        for n in range(0,N):
            Rho12[m,n,:,:] = Rho1[m,n,:,:]@A_dt2 + A_dt1@Rho2[m,n,:,:]
    return Rho12

In [ ]:
def Rho_power_fast(A_dt0,Rho0,power):
    N = A_dt0.shape[0]
    A0 = np.eye(N)
    Rho = Rho0
    for i in range(0,power-1):
        A0 = A0@A_dt0
        Rho = Rho_product_fast(A0,A_dt0,Rho,Rho0)
    return Rho

In [ ]:
def Calc_Rho(transmat_t,R):
    len_t = transmat_t.shape[0]
    Ns = transmat_t.shape[1]
    Rho = np.zeros((len_t,Ns,Ns,Ns,Ns))
    # the follwoing nested for loop is the initiation (eq. 24) step of the h2mm Rho calculation
    for i in range(0,Ns):
        for j in range(0,Ns):
            Rho[0,i,j,i,j] = transmat_t[0,i,j]
    # this is the recursion step
    for t in range(1,len_t):
        R_temp = R[t]
        tempRsize = R_temp.shape[0]
        if tempRsize > 1:
            for Rind in range(0,tempRsize-1):
                if Rind == 0:
                    # this side of the if statement is used to initiate the initiation step
                    transmat_1 = transmat_t[R_temp[Rind,0],:,:]
                    if R_temp[Rind,1] == 1:
                        Rho_1 = Rho[R_temp[Rind,0],:,:,:,:]
                    else:
                        Rho_1 = Rho_product_fast(transmat_1,Rho_1,R_temp[Rind,1])
                        transmat_1 = matrix_power(transmat_1,R_temp[Rind,1])
                    transmat_2 = transmat_t[R_temp[Rind+1,0],:,:]
                    if R_temp[Rind+1,1] == 1:
                        Rho_2 = Rho[R_temp[Rind+1,0],:,:,:,:]
                    else:
                        Rho_2 = Rho_power_fast(transmat_2,Rho_2,R_temp[Rind,1])
                        transmat_2 = matrix_power(transmat_2,Rho_2,R_temp[Rind,1])
                    Rho_temp = Rho_product_fast(transmat_1,transmat_2,Rho_1,Rho_2)
                else:
                    Rho_1 = Rho_temp
                    transmat_1 = transmat_2 @ transmat_2
                    transmat_2 = transmat_t[R_temp[Rind+1,0],:,:]
                    if R_temp[Rind+1,1] == 1:
                        Rho_2 = Rho[R_temp[Rind+1,0],:,:,:,:]
                    else:
                        Rho_2 = Rho_power_fast(transmat_2,Rho[R_temp[Rind+1,0],:,:,:,:],R_temp[Rind+1,1])
                        transmat_2 = matrix_power(transmat_2,R_temp[Rind+1,1])
                    Rho_temp = Rho_product_fast(transmat_1,transmat_2,Rho_1,Rho_2)
            Rho[t,:,:,:,:] = Rho_temp
        else:
            Rho_1 = Rho[R_temp[0,0],:,:,:,:]
            transmat_1 = transmat_t[R_temp[0,0],:,:]
            Rho[t,:,:,:,:] = Rho_power_fast(transmat_1,Rho[R_temp[0,0],:,:,:,:],R_temp[0,1])
    return Rho

In [ ]:
def fwdback_photonByphoton_new(h_model,ph_color,arrivalinds,Rho,transmat_t):
    loglik = 0
    T = arrivalinds.shape[0]
    S = h_model.nstate
    alpha = np.zeros((T,S))
    beta = np.zeros((T,S))
    gamma = np.zeros((T,S))
    xi_summed = np.zeros((S,S))
    obslik = np.zeros((T,S))
    scale = np.zeros(T)
    # Setting up obslik (equivalent to multinomial_prob)
    for t in range(0,T):
        obslik[t,:] = h_model.obs[:,ph_color[t]]
    # alpha initiation
    alpha[0,:] = h_model.prior*obslik[0,:]
    scale[0] = alpha[0,:].sum()
    alpha[0,:] = alpha[0,:]/scale[0]
    # alpha recursion
    for t in range(1,T):
        trans = transmat_t[arrivalinds[t],:,:]
        a = trans.T @ alpha[t-1,:]
        a2 = a * obslik[t,:]
        scale[t] = a2.sum()
        alpha[t,:] = a2 / scale[t]
    if np.any(scale == 0):
        loglik = -np.inf
    else:
        loglik = np.log(scale).sum()
    # beta initiation
    beta[T-1,:] = np.ones(S)
    gamma[T-1,:] = alpha[T-1,:] * beta[T-1,:] / (alpha[T-1,:]*beta[T-1,:]).sum()
    # beta recusion
    for t in range(T-2,-1,-1):
        b = beta[t+1,:]*obslik[t+1,:]
        trans = transmat_t[arrivalinds[t+1],:,:]
        trans0 = trans
        trans0[trans0==0] = 1
        beta[t,:] = (trans @ b) / (trans @ b).sum()
        gamma[t,:] = alpha[t,:]*beta[t,:]/(alpha[t,:]*beta[t,:]).sum()
        xi_temp = trans * np.outer(alpha[t,:],b)
        xi_temp = xi_temp/xi_temp.sum()
        fxi = ((xi_temp / trans0)* Rho[arrivalinds[t+1],:,:,:,:]).sum(axis=3).sum(axis=2)
        xi_summed += fxi
    return fwdback_values(alpha,beta,gamma,loglik,xi_summed,scale)

In [ ]:
def compute_ess_dhmm(h_model,ph_color,ArrivalInds,R):
    numex = len(ArrivalInds)
    Nd = h_model.obs.shape[1]
    exp_num_trans = np.zeros((h_model.nstate,h_model.nstate))
    exp_num_visits = np.zeros(h_model.nstate)
    exp_num_emits = np.zeros((numex,h_model.nstate,Nd))
    loglik = 0
    sc = np.zeros((len(ArrivalInds)))
    transmat_t = CalculatePowerofTransMatrices(h_model,R.R)
    Rho = Calc_Rho(transmat_t,R.R)
    for ex in range(0,numex):
        T = ph_color[ex].size
        param_temp = fwdback_photonByphoton_new(h_model,ph_color[ex],ArrivalInds[ex],Rho,transmat_t)
        loglik += param_temp.loglik
        exp_num_trans += param_temp.xi_summed
        exp_num_visits += param_temp.gamma[0,:]
        if T < Nd:
            for t in range(0,T):
                p = ph_color[ex][t]
                exp_num_emits[ex,p,:] += param_temp.gamma[t,:]
                print('Triggered T<Ns')
        else:
            for n in range(0,Nd):
                ndx = np.argwhere(ph_color[ex]==n)[:,0]
                if ndx.size != 0:
                    exp_num_emits[ex,:,n] = param_temp.gamma[ndx,:].sum(axis=0)
    exp_num_emit = exp_num_emits.sum(axis=0)
    return h2mm_model(exp_num_visits,exp_num_trans,exp_num_emit,loglik=loglik)

In [ ]:
def EM_H2MM(h_mod,burst_colors,burst_times,max_iter=3600,max_time=np.inf,converged_min=1e-14):
    assert len(burst_times) == len(burst_colors)
    TotalArrivalDelta = np.empty((0),dtype=int)
    burst_deltas = []
    for i in range(0,len(burst_times)):
        assert burst_colors[i].size == burst_times[i].size
        deltas_temp = np.diff(burst_times[i])
        TotalArrivalDelta = np.append(TotalArrivalDelta,deltas_temp)
        burst_deltas.append(np.append(1,deltas_temp))
    assert len(burst_deltas) == len(burst_times)
    R = ph_factors(TotalArrivalDelta)
    for i in range(0,len(burst_deltas)):
        burst_inds.append(np.ones(burst_deltas[i].shape,dtype=int))
        for t in range(0,burst_deltas[i].size):
            burst_inds[i][t] = np.argwhere(burst_deltas[i][t] == R.deltas)[0,0]
        assert burst_inds[i].size == burst_colors[i].size
    assert len(burst_inds) == len(burst_colors)
    
    # Initialize variable for main optimization loop
    h_mod_current = h_mod
    LL = np.empty((0),dtype=float)
    niter = 0
    cont = True
    tm_start = time.perf_counter()
    tm_iter = tm_start
    print('Begin compute_ess_dhmm')
    while cont: # while loop loops until one of the end conditions is met...
        h_mod_temp = compute_ess_dhmm(h_mod,burst_colors,burst_inds,R) # calculates new model
        if h_mod_temp.loglik - h_mod_current.loglik > converged_min: # check main condition of convergence
            h_mod_current = h_mod_temp
            LL = np.append(LL,h_mod_current.loglik)
            tm_prev = tm_iter
            tm_iter = time.perf_counter()
            tm_elapse = tm_iter - tm_start
            tm_cycle = tm_iter - tm_prev
            print(f'Iteration {n}, loglik {h_mod_current.loglik}, iteration time {tm_cycle}, total time {tm_elapse}')
            n += 1
            if max_iter < n: # checking alternative end conditions, 
                h_mod_current = h_mod_temp
                print('Maximum iterations reached')
                cont = False
            if max_time*3600 < tm_elapse:
                print('Maximum time reached')
                cont = False
        else:
            print('Converged')
            cont = False
    return h_mod_current

In [ ]:
def viterbi_path_PhotonByPhoton(h_model,ph_color,ph_arrivaltime):
    assert ph_color.size == ph_arrivaltime.size
    T = ph_color.shape[0]
    S = h_model.nstate
    obslik = np.zeros(T,S)
    for t in range(0,T):
        obslik[t,:] = h_model.obs[:,ph_color[t]]
    ph_arrivaldiff = np.diff(ph_arrivaltime)
    ph_arrivaldiff_max = np.max(ph_arrivaldiff)
    transmat_t = np.zeros((ph_arrivaldiff_max,S,S))
    transmat_t[0,:,:] = h_model.trans
    for t in range(1,ph_arrivaldiff_max):
        transmat_t[t,:,:] = transmat_t[t-1,:,:] @ h_model.trans
    delta = np.zeros((T,S))
    psi = np.zeros((T,S))
    obslik = np.zeros((T,S))
    scale = np.ones(T)
    path = np.zeros(T,dtype=int)
    t = 0
    delta[t,:] = h_model.prior * obslik[t,:]
    scale[t] = 1/delta[t,:].sum()
    delta[t,:] = delta[t,:]/delta[t,:].sum()
    psi[t,:] = 0
    for t in range(1,T):
        trans = transmat_t[ph_arrivaldiff[t-1],:,:]
        for j in range(0,S):
            delta_temp = trans[:,j] * delta[t-1,:]
            delta[t,j] = np.max(delta_temp)
            psi[t,j] = np.argwhere(delta[t,j]==delta_temp)[0,0]
            delta[t,j] = delta[t,j] * obslik[t,j]
        scale[t] = 1/delta[t,:]
        delta[t,:] = delta[t,:]/delta[t,:].sum()
    t = T-1
    path[t] = np.argwhere(np.max(delta[t,:])==delta[t,:])[0,0]
    for t in range(T-2,-1,-1):
        path[t] = psi[t+1,path[t+1]]
    return path

First we isolate and filter the photon stream for only photon during the donor excitation period
2 masks are made in the end, one for all the photons and one for the acceptor photons
The final for loop creates the interphoton time list and isolates the bursts into sepeparate cells in the list

# Load Data

In [ ]:
filename = '/home/paul/Documents/FRETBursts_notebooks-master/data/2019-12-23-Nick.sptw/Nick0-1MgCl10KCl2.hdf5'
if os.path.isfile(filename):
    print('File found, you can proceed to load the file')
else:
    print("Sorry, I can't find the file \n%s" % filename)

In [ ]:
d = loader.photon_hdf5(filename)

# Alex and Background processing

In [ ]:
bpl.plot_alternation_hist(d)

In [ ]:
loader.alex_apply_period(d)

In [ ]:
d.calc_bg(fun=bg.exp_fit,time_s=30, tail_min_us='auto', F_bg=1.7)

dplot(d, hist_bg)

In [ ]:
dplot(d, timetrace)

# Performing Burst Search

In [ ]:
d.burst_search(m=10, F=6, ph_sel=Ph_sel(Dex='DAem'))
d.fuse_bursts(ms=0)

alex_jointplot(d)

In [ ]:
d_all = Sel(d, select_bursts.size, th1=50)
alex_jointplot(d_all)

In [ ]:
d_all = Sel(d_all, select_bursts.naa, th1=50)
alex_jointplot(d_all)

# Processing Real Data

In [ ]:
print('Defining photon masks')
DAemDex_mask = d_all.get_ph_mask(ph_sel=Ph_sel(Dex='DAem'))
AemDex_mask = d_all.get_ph_mask(ph_sel=Ph_sel(Dex='Aem'))
AemDex_mask_red = AemDex_mask[DAemDex_mask]

d_all_bursts = d_all.mburst[0]
ph_d = d_all.get_ph_times(ph_sel=Ph_sel(Dex='DAem'))
print('Reducing photon burst indexes')
burst_red = d_all_bursts.recompute_index_reduce(ph_d)

TotalArrivalDelta = np.empty([1,0], dtype=int)
ArrivalMask = []
BurstArrivalDelta = []
ArrivalColor = []

print('Calculating interphoton times')
for start, stop in zip(burst_red.istart, burst_red.istop):
    ArrivalDeltaTemp = np.diff(ph_d[start:stop+1])
#    ArrivalDeltaTemp[ArrivalDeltaTemp==0] = 1 # removes 0 delta times from the list... think through this more
    TotalArrivalDelta = np.append(TotalArrivalDelta, ArrivalDeltaTemp)
    BurstArrivalDelta.append(np.append(0, ArrivalDeltaTemp))
    ArrivalMask.append(AemDex_mask_red[start:stop+1])
    ArrivalColor_temp = np.zeros(AemDex_mask_red[start:stop+1].size,dtype=int)
    ArrivalColor_temp[AemDex_mask_red[start:stop+1]] = 1
    ArrivalColor.append(ArrivalColor_temp)

Now that the table is made, create an array of the burst with the indexes of the appropriate element of the look up table cooresponding to the interphoton time

In [ ]:
R = ph_factors(TotalArrivalDelta)

ArrivalTimeInds = []
for i in range(0,len(BurstArrivalDelta)):
    ArrivalTimeInds.append(np.array([],dtype=int))
    for j in range(0,len(BurstArrivalDelta[i])):
        if BurstArrivalDelta[i][j] == 0:
            ArrivalTimeInds[i] = np.append(ArrivalTimeInds[i],np.argwhere(1==R.deltas))
        else:
            ArrivalTimeInds[i] = np.append(ArrivalTimeInds[i],np.argwhere(BurstArrivalDelta[i][j]==R.deltas))

In [ ]:
prior2 = np.array([0.1, 0.9])
trans2 = np.array([[0.998, 0.002],[0.0001, 0.9999]])
obs2 = np.array([[0.3, 0.7],[0.8, 0.2]])
h_mod = h2mm_model(prior2,trans2,obs2)
h_mod.normalize()
h_mod2_i = h_mod

transmat_t = CalculatePowerofTransMatrices(h_mod,R.R)
Rho = Calc_Rho(transmat_t,R.R)

#param_temp = []
#for ex in range(0,len(ArrivalColor)):
#    param_temp.append(fwdback_photonByphoton_new(h_mod,ArrivalColor[ex],ArrivalTimeInds[ex],Rho,transmat_t))


In [ ]:
niter = 0
LL2 = []
max_iter = 3600
cont = True
print('Begin compute_ess_dhmm')
tm_start = time.perf_counter()
tm_iter = tm_start
while cont:
    h_mod_temp = compute_ess_dhmm(h_mod,ArrivalColor,ArrivalTimeInds,R)
    if h_mod_temp.loglik < h_mod.loglik:
        h_mod2 = h_mod
        cont = False
        print(f'Converged old loglik {h_mod.loglik}, new loglik {h_mod_temp.loglik}')
    else:
        LL2 = np.append(LL2,h_mod_temp.loglik)
        h_mod_temp.normalize()
        h_mod = h_mod_temp
        # the terms are just keeping track of the time elapsed
        tm_prev = tm_iter
        tm_iter = time.perf_counter()
        tm_elapse = tm_iter - tm_start
        tm_cycle = tm_iter - tm_prev
        # print the iteration number
        print(f'Iteration {niter}, loglik {LL2[-1]}, iteration time: {tm_cycle}, total time: {tm_elapse}')
        niter += 1
        if niter > max_iter:
            cont = False

In [ ]:
prior3 = np.array([0.3, 0.5, 0.2])
trans3 = np.array([[0.9998, 0.0001, 0.0001],[0.00001, 0.99998, 0.00001],[0.000001, 0.000001, 0.999998]])
obs3 = np.array([[0.1, 0.9],[0.2, 0.8],[0.3, 0.7]])
h_mod = h2mm_model(prior3,trans3,obs3)
h_mod3_i = h_mod

In [ ]:
niter = 0
LL3 = []
max_iter = 3600
cont = True
print('Begin compute_ess_dhmm')
tm_start = time.perf_counter()
tm_iter = tm_start
while cont:
    h_mod_temp = compute_ess_dhmm(h_mod,ArrivalColor,ArrivalTimeInds,R)
    if h_mod_temp.loglik < h_mod.loglik:
        h_mod3 = h_mod
        cont = False
        print(f'Converged old loglik {h_mod.loglik}, new loglik {h_mod_temp.loglik}')
    else:
        LL3 = np.append(LL3,h_mod_temp.loglik)
        h_mod_temp.normalize()
        h_mod = h_mod_temp
        # the terms are just keeping track of the time elapsed
        tm_prev = tm_iter
        tm_iter = time.perf_counter()
        tm_elapse = tm_iter - tm_start
        tm_cycle = tm_iter - tm_prev
        # print the iteration number
        print(f'Iteration {niter}, loglik {LL3[-1]}, iteration time: {tm_cycle}, total time: {tm_elapse}')
        niter += 1
        if niter > max_iter:
            cont = False

In [ ]:
h_mod2.obs

In [ ]:
h_mod2.trans

In [ ]:
h_mod2.prior

In [ ]:
scale = []
xi_summed = []
gamma = []
alpha = []
beta = []
with h5py.File('/home/paul/Octave/jp6b10726_si_002/real_data.hdf5') as hdf:
    scalef = hdf.get('scale/value')
    xi_summedf = hdf.get('xi_summed/value')
    gammaf = hdf.get('gamma/value')
    alphaf = hdf.get('alpha/value')
    
    ls = list(scalef.keys())
    for key in ls:
        scale_temp = scalef.get(key + '/value')
        scale.append(np.array(scale_temp))
        xi_summed_temp = xi_summedf.get(key+'/value')
        xi_summed.append(np.array(xi_summed_temp))
        gamma_temp = gammaf.get(key+'/value')
        gamma.append(np.array(gamma_temp))
        alpha_temp = alphaf.get(key+'/value')
        alpha.append(np.array(alpha_temp))

In [ ]:
gamma = gamma[0:4383]
scale = scale[0:4383]
xi_summed = xi_summed[0:4383]

In [ ]:
scalenp = np.empty((0),dtype=float)
for i in range(0,len(scale)):
    scalenp = np.append(scalenp,scale[i])

In [ ]:
scalepy = np.empty((0),dtype=float)
for i in range(0,len(param_temp)):
    scalepy = np.append(scalepy,np.log(param_temp[i].scale).sum())

In [ ]:
xi_summedml = np.zeros((2,2))
xi_summedpy = np.zeros((2,2))
for i in range(0,4383):
    xi_summedml += xi_summed[i]
    xi_summedpy += param_temp[i].xi_summed
print(xi_summedml)
print(xi_summedpy)

# Test Data Construction

In [ ]:
trans2 = np.array([[0.9999,0.0001],[0.001,0.999]])
prior2 = np.array([0.3,0.7])
obs2 = np.array([[0.1,0.9],[0.4,0.6]])
h_mod2 = h2mm_model(prior2,trans2,obs2)
trans3 = np.array([[0.98, 0.01, 0.01],[0.01, 0.98, 0.01],[0.01, 0.01, 0.98]])
prior3 = np.array([0.2, 0.6, 0.2])
obs3 = np.array([[0.1, 0.9],[0.5, 0.5],[0.7, 0.3]])
h_mod3 = h2mm_model(prior3, trans3, obs3)
data_time_short = np.zeros(12,dtype=int)

for i in range(1,12):
    if i != 4:
        data_time_short[i] = i + data_time_short[i-1]
    else:
        data_time_short[i] = i + data_time_short[i-1] + 3

data_time_short = np.array([0, 4, 6, 13, 19, 21, 50, 54, 55, 59, 70, 72])
data_ph_short = np.zeros(12,dtype=int)
for i in range(1,len(data_time_short)):
    data_ph_short[i] = data_time_short[i] % 2

deltas_short = np.diff(data_time_short)
R_short = ph_factors(deltas_short)
deltas_short = np.diff(data_time_short)
R_short = ph_factors(deltas_short)
transmat_t_short = CalculatePowerofTransMatrices(h_mod2,R_short.R)
transmat_t3_short = CalculatePowerofTransMatrices(h_mod3,R_short.R)

In [ ]:
Rho_short = Calc_Rho(transmat_t_short,R_short.R)
Rho3_short = Calc_Rho(transmat_t3_short,R_short.R)

In [ ]:
arrivalinds_short = np.zeros((deltas_short.shape[0]+1),dtype=int)
for i in range(0,deltas_short.shape[0]):
    arrivalinds_short[i+1] = np.argwhere(deltas_short[i]==R_short.deltas)

In [ ]:
param_temp2_short = fwdback_photonByphoton_new(h_mod2,data_ph_short,arrivalinds_short,Rho_short,transmat_t_short)

In [ ]:
param_temp3_short = fwdback_photonByphoton_new(h_mod3,data_ph_short,arrivalinds_short,Rho3_short,transmat_t3_short)

In [ ]:
param_temp2_short.xi_summed

In [ ]:
param_temp3_short.xi_summed

In [ ]:
h_mod2.trans

In [ ]:
e_n_trans_short = np.zeros(param_temp2_short.xi_summed.shape)
for i in range(0,2):
    e_n_trans_short[i,:] = param_temp2_short.xi_summed[i,:]/param_temp2_short.xi_summed[i,:].sum()
e_n_prior_short = param_temp2_short.gamma[0,:]/param_temp2_short.gamma[0,:].sum()

In [ ]:
e_n_emits_short = np.zeros((2,2))
for i in range(0,2):
    ndx = np.argwhere(data_ph_short == i)[:,0]
    if ndx.size > 0:
        e_n_emits_short[:,i] = param_temp2_short.gamma[ndx,:].sum(axis=0)
for i in range(0,2):
    e_n_emits_short[i,:] = e_n_emits_short[i,:]/e_n_emits_short[i,:].sum()

In [ ]:
test11 = np.array([[1,2],[3,4]]).T
test21 = np.array([[5,6],[7,8]]).T
test12 = np.array([[9,10],[11,12]]).T
test22 = np.array([[13,14],[15,16]]).T

test = np.array([[test11,test12],[test21,test22]])

In [ ]:
cont = True
nit = 0
h_mod_current_short = h_mod2
while cont:
    nit += 1
    h_mod_temp_short = compute_ess_dhmm(h_mod_current_short,[data_ph_short],[arrivalinds_short],R_short)
    if h_mod_temp_short.loglik > h_mod_current_short.loglik:
        h_mod_current_short = h_mod_temp_short
        h_mod_current_short.normalize()
        print(f'{h_mod_current_short.loglik} nit = {nit}')
    else:
        cont = False

In [ ]:
# keeping old broken verions of the Calc_Rho function around

#def Calc_Rho(transmat_t,R):  # this has problems in the calculation- check the if/for loops (most complicated)
#    len_t = transmat_t.shape[0]
#    transmat = transmat_t[0,:,:]
#    Ns = transmat.shape[0]
#    Rho = np.zeros((len_t,Ns,Ns,Ns,Ns)) # Rho is now a 5D matrix,with .shape[0] dimention replacing the cell array dimention in the matlab function
#    for i in range(0,Ns):
#        for j in range(0,Ns):
#            Rho[0,i,j,i,j] = transmat[i,j]
#    for t in range(1,len_t):
#        Rtemp = R[t]
#        tempRsize = Rtemp.shape[0]
#        if tempRsize > 1:
#            for Rind in range(0,tempRsize-1):
#                if Rind == 0:
#                    transmat_1 = transmat_t[Rtemp[Rind,0],:,:]
#                    if Rtemp[Rind,1] == 1:
#                        Rho_1 = Rho[Rtemp[Rind,0],:,:,:,:]
#                    else:
#                        Rho_1 = Rho_power_fast(transmat_1,Rho[Rtemp[Rind,0],:,:,:,:],Rtemp[Rind,1])
#                        transmat_1 = matrix_power(transmat_1,Rtemp[Rind,1])
#                    transmat_2 = transmat_t[Rtemp[Rind,0],:,:]
#                    if Rtemp[Rind+1,1] == 1:
#                        Rho_2 = Rho[Rtemp[Rind+1,0],:,:,:,:]
#                    else:
#                        Rho_2 = Rho_power_fast(transmat_2,Rho[Rtemp[Rind+1,0],:,:,:,:],Rtemp[Rind+1,1])
#                        transmat_2 = matrix_power(transmat_2,Rtemp[Rind+1,1])
#                    Rho_temp = Rho_product_fast(transmat_1,transmat_2,Rho_1,Rho_2)
#                else:
#                    Rho_1 = Rho_temp
#                    transmat_1 = transmat_1 @ transmat_2
#                    transmat_2 = transmat_t[Rtemp[Rind+1,0],:,:]
#                    if Rtemp[Rind+1,1] == 1:
#                        Rho_2 = Rho[Rtemp[Rind+1,0],:,:,:,:]
#                    else:
#                        Rho_2 = Rho_power_fast(transmat_2,Rho[Rtemp[Rind+1,0],:,:,:,:],Rtemp[Rind+1,1])
#                        transmat_2 = matrix_power(transmat_2,Rtemp[Rind+1,1])
#                    Rho_temp = Rho_product_fast(transmat_1,transmat_2,Rho_1,Rho_2)
#            Rho[t,:,:,:,:] = Rho_temp
#        else:
#            Rho1 = Rho[Rtemp[0,0],:,:,:,:]
#            transmat_1 = transmat_t[Rtemp[0,0],:,:]
#            Rho[t,:,:,:,:] = Rho_power_fast(transmat_1,Rho1,Rtemp[0,1])
#    return Rho

In [ ]:
## This function has been deprecated
#
#
#class photon_tbl:
#    def __init__(self,deltas,R=[np.array([0,1],dtype=int)]):
#        self.deltas = (deltas) # equivalent to Augmented_T in matlab Factorization function
#        self.R = (R) # equivalent to R in in matlab Factorization function
#def Factorization(Deltas): # this is slow when the number of factors is small, prime target for cython
#    class photon_lst:
#        def __init__(self,deltas,R):
#            self.deltas = np.unique(deltas)
#            self.R = R
#        def factor(self): # for malab equivalent, see Main_rutin in the Factorization function
#            deltas = np.unique(self.deltas)
#            self.R = [np.array([[0,1]],dtype=int)]
#            deltas = deltas[deltas != 0]
#            if deltas[0] != 1:
#                deltas = np.append(1,deltas)
#            for i in range(1,deltas.size):
#                d = deltas[i] // deltas[i-1]
#                m = deltas[i] % deltas[i-1]
#                self.R.append(np.array([[i-1,d]]))
#                while m != 0:
#                    k_arg = np.argwhere(deltas[:i-1]==m)
#                    if 0 == k_arg.size:
#                        k = np.argwhere(deltas[:i-1]<m)[-1,0]
#                        d = m // deltas[k]
#                        m = m % deltas[k]
#                        temp_factors = np.array([[k,d]])
#                        self.R[i] = np.concatenate((self.R[i],temp_factors))
##                        if d > 1 and k > 0:
##                            print(f'Iteration {i}, k = {k}, d = {d}, m = {m}, deltas = {self.deltas[k]}')
#                    else:
#                        temp_factors = np.array([[k_arg[0][0],1]])
#                        self.R[i] = np.concatenate((self.R[i],temp_factors))
#                        m = 0
#    lkuptbl = photon_lst(Deltas,np.array([0,1]))
#    lkuptbl.factor()
#    B = np.array([],dtype=int)
#    for i in range(1,len(lkuptbl.R)):
#        if lkuptbl.R[i].shape[0] > 1:
#            k = np.argwhere(lkuptbl.R[i][:,1]>1)[:,0]
#            if k.size > 1:
#                for j in range(0,k.size):
#                    temp_ind = k[j]
#                    temp_delta = lkuptbl.deltas[k] * lkuptbl.R[i][k,1]
#                    B = np.append(B,temp_delta)
#    B = np.unique(B)
#    if B.size != 0:
#        deltas_temp = np.append(lkuptbl.deltas,B)
#        print(deltas_temp)
#        lkuptbl.deltas = np.unique(deltas_temp)
#        lkuptbl.factor()
#    return photon_tbl(lkuptbl.deltas, lkuptbl.R)

In [ ]:
# This cell was the original version of the factorization function, it does not generate the deltas output
#def Factorization1(Deltas):
#    Deltas = np.unique(Deltas)
#    if np.any(Deltas==0):
#        print('Removing interphoton times with delta of 0')
#        Deltas = Deltas[Deltas!=0]
#    if Deltas[0] != 1:
#        Deltas = np.append(1, Deltas)
#    print('Begin factorization proccess')
#    R = [np.array([0,1])]
#    B = np.array([],dtype='int32')
#    for i in range(1,len(Deltas)):
#        d = Deltas[i] // Deltas[i-1]
#        m = Deltas[i] % Deltas[i-1]
#        R.append(np.array([i-1,d]))
#        while m != 0:
#            if ~np.any(Deltas[:i-1]==m):
#                k = np.argwhere(Deltas[:i-1]<m)[-1]
#                d = m // Deltas[k]
#                m = m % Deltas[k]
#                R[i] = np.stack((R[i],np.array([k[0],d])),axis=0)
#                if d > 1:
#                    B = np.append(B,Deltas[k[0]]*d)
#            else:
#                ind = np.argwhere(Deltas[:i-1]==m)[0]
#                R[i] = np.stack((R[i],np.array([ind[0],1])),axis=0)
#                m = 0
#    if len(B) > 0:
#        Deltas = np.append(B,Deltas)
#        Deltas = np.unique(Deltas)
#        R = Factorization(Deltas)
#    return R

In [ ]:
#def fwdback_photonByphoton_fast(h_model,pcolor,delta_t,flux_free,transmat_t):
#    loglik = 0
#    T = pcolor.size
#    alpha = np.zeros((T,h_model.nstate))
#    gamma = np.zeros((T,h_model.nstate))
#    xi_summed = np.zeros((h_model.nstate,h_model.nstate))
#    # set up obslik, equivalent to the multinomial_prob function in matlab
#    obslik = np.zeros((T,h_model.nstate))
#    scale = np.zeros(T)
#    for t in range(0,T):
#        obslik[t,:] = h_model.obs[:,pcolor[t]]
#    #forward variable
#    t = 0
#    alpha[t,:] = h_model.prior * obslik[t,:]
#    scale[t] = alpha[t,:].sum()
#    alpha[t,:] = alpha[t,:] / scale[t]
#    for t in range(1,T):
#        trans = transmat_t[delta_t[t],:,:]
#        m = trans.T @ alpha[t-1,:]
#        alpha[t,:] = m * obslik[t,:]
#        scale[t] = alpha[t,:].sum()
#        alpha[t,:] = alpha[t,:] / scale[t]
#    if np.any(scale == 0):
#        loglik = -np.inf
#    else:
#        loglik = np.sum(np.log(scale))
#    # backward variable
#    beta = np.zeros((T,h_model.nstate))
#    beta[T-1,:] = np.ones(h_model.nstate)
#    gamma[T-1,:] = (alpha[T-1,:]*beta[T-1,:])/np.sum(alpha[T-1,:]*beta[T-1,:])
#    for t in range(T-2,-1,-1):
#        b = beta[t+1,:]*obslik[t+1,:]
#        print(b)
#        trans = transmat_t[delta_t[t],:,:]
#        trans_temp = trans
#        trans_temp[trans_temp==0] = 1
#        beta[t,:] = trans @ b
#        beta[t,:] = beta[t,:]/beta[t,:].sum()
#        gamma[t,:] = (alpha[t,:]*beta[t,:])/np.sum(alpha[t,:]*beta[t,:])
#        xi_temp = (trans @ np.outer(alpha[t,:],b))/np.sum(trans @ np.outer(alpha[t,:],b)) # something is wrong with this calculation
#        fxi = ((xi_temp/trans_temp)*flux_free[delta_t[t+1],:,:,:,:]).sum(axis=3).sum(axis=2)
#        xi_summed += fxi
#    return fwdback_values(alpha,beta,gamma,loglik,xi_summed)